<div style="text-align: center; border: 2px solid #3498db; padding: 10px; background-color: #192a56;">
    <h1 style="color: #ff6b6b;">Recommendation System</h1>
</div>

<h2 style="color: #3498db;">IMPORT LIBRARIES</h2>

In [1]:
import numpy as np
import pandas as pd

<h2 style="color: #e74c3c;">Import datasets</h2>

In [2]:
similarity = pd.read_csv("Product-Product Similarity.csv")
scores = pd.read_csv("Product score.csv")[["product_id","Product name","Overall score"]].set_index("product_id")
groups = pd.read_csv("Groups.csv")
users = pd.read_csv("User groups.csv",index_col=0)
products = pd.read_csv("product_info.csv")

C:\Users\rkros\AppData\Local\Temp\ipykernel_7112\3369692860.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv("User groups.csv",index_col=0)


<h2 style="color: #27ae60;">Preprocessing</h2>

In [3]:
similarity.set_index("product_name",inplace=True)

In [4]:
similarity = similarity[similarity.index.drop_duplicates()].T

In [5]:
users["Product score"] = pd.merge(users,scores,on="product_id")["Overall score"]

In [6]:
groups

,Unnamed: 0,Skin tone,Eye color,Skin type,Hair color,Group name
0,0,medium,brown,oily,black,0
1,1,medium,brown,oily,brown,1
2,2,medium,brown,oily,blonde,2
3,3,medium,brown,oily,auburn,3
4,4,medium,brown,oily,gray,4
...,...,...,...,...,...,...
1815,1815,ebony,blue,normal,blonde,12432
1816,1816,ebony,blue,normal,auburn,12433
1817,1817,ebony,blue,normal,gray,12434
1818,1818,ebony,blue,normal,red,12435


<h2 style="color: #e67e22;">Creating recommendations for each group</h2>

In [7]:
recommendations = pd.DataFrame(index=groups["Group name"],columns=["Recommendations"])

In [8]:
for i in recommendations.index:
    products_list = users[(users["Group name"]==i) & (users["Product score"]>50)][["product_name","Product score"]]
    recommendations.loc[i,"Recommendations"]=products_list.drop_duplicates().sort_values("Product score",ascending=False).iloc[:15].product_name.values

In [13]:
recommendations

,Recommendations
Group name,
0,[Lip Sleeping Mask Intense Hydration with Vita...
1,[Lip Sleeping Mask Intense Hydration with Vita...
2,[Lip Sleeping Mask Intense Hydration with Vita...
3,[Lip Sleeping Mask Intense Hydration with Vita...
4,"[Lip Glowy Balm, Green Clean Makeup Removing C..."
...,...
12432,[]
12433,[]
12434,[]


In [9]:
recommendations.Recommendations.apply(lambda a : len(a)).value_counts()

15    695
0     655
1     119
2      65
3      50
6      38
4      36
5      29
7      24
8      21
10     21
9      17
13     15
12     12
11     12
14     11
Name: Recommendations, dtype: int64

**Fill recommendations based on relevant groups with same skin type, skin color and eye color but with different hair color**

In [9]:
for i in recommendations.index:
    if len(recommendations.Recommendations.loc[i])<15:
        remaining = 15 - len(recommendations.Recommendations.loc[i])
        group = str(i).rjust(5,"0")[:4]
        for j in range(7):
            group_name = int(group+str(j))
            if len(recommendations.Recommendations.loc[group_name])>=remaining:
                recommendations.Recommendations.loc[i] = np.concatenate((recommendations.Recommendations.loc[i],recommendations.Recommendations.loc[group_name][:remaining]))
                break

**Fill recommendations based on relevant groups with same skin type, hair color and skin color but with different eye color**

In [10]:
for i in recommendations.index:
    if len(recommendations.Recommendations.loc[i])<15:
        remaining = 15 - len(recommendations.Recommendations.loc[i])
        group = str(i).rjust(5,"0")
        for j in range(5):
            group_name = int(group[:2]+str(j)+group[3:])
            if len(recommendations.Recommendations.loc[group_name])>=remaining:
                recommendations.Recommendations.loc[i] = np.concatenate((recommendations.Recommendations.loc[i],recommendations.Recommendations.loc[group_name][:remaining]))
                break

**Fill recommendations based on relevant groups with same skin type, hair color and eye color but with different skin color**

In [11]:
for i in recommendations.index:
    if len(recommendations.Recommendations.loc[i])<15:
        remaining = 15 - len(recommendations.Recommendations.loc[i])
        group = str(i).rjust(5,"0")
        for j in range(13):
            group_name = int(str(j).rjust(2,"0")+group[2:])
            if len(recommendations.Recommendations.loc[group_name])>=remaining:
                recommendations.Recommendations.loc[i] = np.concatenate((recommendations.Recommendations.loc[i],recommendations.Recommendations.loc[group_name][:remaining]))
                break

In [12]:
recommendations.Recommendations.apply(lambda a : len(a)).value_counts()

15    1820
Name: Recommendations, dtype: int64

**Fill recommendations based on similar products using product similarity matrix**

In [13]:
for i in recommendations.index:
    similarity_recommendations = []
    for j in recommendations.loc[i].Recommendations:
        similarity_recommendations.append(similarity.loc[j].sort_values(ascending=False).index[1])
    recommendations.loc[i].Recommendations = np.concatenate((recommendations.loc[i].Recommendations,similarity_recommendations))

In [14]:
recommendations.Recommendations.apply(lambda a : len(a)).value_counts()

30    1820
Name: Recommendations, dtype: int64

In [15]:
recommendations.to_csv("Product_Recommendations.csv")

<h2 style="color: #9b59b6;">User Interface</h2>

In [5]:
pip install customtkinter

     -------------------------------------- 295.6/295.6 KB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'D:\PGP-DSE\Python\Anaconda\python.exe -m pip install --upgrade pip' command.


In [22]:
skin_tones = groups["Skin tone"].unique()
skin_types = groups["Skin type"].unique()
eye_colors = groups["Eye color"].unique()
hair_colors = groups["Hair color"].unique()

In [ ]:
import customtkinter
import tkinter as tk

In [30]:
def user_page():
    
    root.geometry("500x500")
    
    # Fetch user name
    user_name = name.get()
    
    # Define global variables
    global skin_tone_options
    global skin_type_options
    global eye_color_options
    global hair_color_options
    global gender_options 
    
    # Clear the screen
    for widget in frame.winfo_children():
        widget.destroy()
    
    greetings = customtkinter.CTkLabel(master=frame,text=f"Hello, {user_name}!\nA very warm WELCOME!\n It is lovely to have you!\n Please enter your details",font=("Roboto",24))
    greetings.pack(pady=12,padx=10)
    
    gender = customtkinter.CTkLabel(master=frame, text="Gender:")
    gender.place(x=80,y=150)
    
    gender_options = customtkinter.CTkSegmentedButton(master=frame,values=["Male","Female"])
    gender_options.place(x=160,y=150)
    
    skin_tone = customtkinter.CTkLabel(master=frame, text="Skin tone:")
    skin_tone_options = customtkinter.CTkOptionMenu(master=frame, values=skin_tones)
    
    skin_type = customtkinter.CTkLabel(master=frame, text="Skin type:")    
    skin_type_options = customtkinter.CTkOptionMenu(master=frame, values=skin_types)
    
    eye_color = customtkinter.CTkLabel(master=frame, text="Eye color:", anchor="w")
    eye_color_options = customtkinter.CTkOptionMenu(master=frame, values=eye_colors)

    hair_color = customtkinter.CTkLabel(master=frame, text="Hair color:", anchor="w") 
    hair_color_options = customtkinter.CTkOptionMenu(master=frame, values=hair_colors)

    labels = [skin_tone,skin_type,eye_color,hair_color]
    option_menus = [skin_tone_options,skin_type_options,eye_color_options,hair_color_options]

    for i in range(len(labels)):
        labels[i].place(x=80, y=200+(i*50))
        option_menus[i].place(x=160, y=200+(i*50))
        
    def gender_check():
        
        # check for men
        if (gender_options.get()=="Male"):
            
            # Clear the screen
            for widget in frame.winfo_children():
                widget.destroy()
            
            male_text = customtkinter.CTkLabel(master=frame,text="Sorry!\n There are no product for men\n in our website\n Try searching for any other good website :)",font=("Roboto",18))
            male_text.pack(pady=12,padx=10)
            return
        else:
            filter_page()
        
        
    button = customtkinter.CTkButton(master=frame,text="Continue",command=gender_check)
    button.place(x=100,y=400)

In [29]:
def filter_page():
    
    # Define global variables to store attributes
    global skin_tone_label
    global skin_type_label
    global eye_color_label
    global hair_color_label
    
    # Fetch user attributes
    skin_tone_label = [k for k,v in dict(pd.Series(skin_tones)).items() if v==skin_tone_options.get()]
    skin_type_label = [k for k,v in dict(pd.Series(skin_types)).items() if v==skin_type_options.get()]
    eye_color_label = [k for k,v in dict(pd.Series(eye_colors)).items() if v==eye_color_options.get()]
    hair_color_label = [k for k,v in dict(pd.Series(hair_colors)).items() if v==hair_color_options.get()]
    
    # Clear the screen
    for widget in frame.winfo_children():
        widget.destroy()
    
    # Global variables
    global secondary_switch
    global tertiary_switch
    
    secondary_category_text = customtkinter.CTkLabel(master=frame,text="Do you want to apply filter on products\n based on secondary categories ?",font=("Roboto",20))
    secondary_category_text.pack(pady=12,padx=10)
    
    secondary_filter = customtkinter.CTkLabel(master=frame,text="Secondary Category filter",font=("Roboto",16))
    secondary_filter.place(x=40,y=80)
    
    tertiary_category_text = customtkinter.CTkLabel(master=frame,text="Do you want to apply filter on products\n based on tertiary categories ?",font=("Roboto",20))
    tertiary_category_text.place(x=24,y=160)
    
    tertiary_filter = customtkinter.CTkLabel(master=frame,text="Tertiary Category filter",font=("Roboto",16))
    tertiary_filter.place(x=40,y=230)
    
    button = customtkinter.CTkButton(master=frame,text="Continue",command=recommendation_page)
    button.place(x=100,y=350)
    
    secondary_switch = customtkinter.CTkSwitch(master=frame, text="",command=secondary_category_filter)
    secondary_switch.place(x=230,y=82)
    
    tertiary_switch = customtkinter.CTkSwitch(master=frame, text="",command=tertiary_category_filter)
    tertiary_switch.place(x=230,y=232)

def secondary_category_filter():
    
    global secondary_category_options
    
    # Group name
    group_name = int(str(skin_tone_label[0])+str(skin_type_label[0])+str(hair_color_label[0])+str(eye_color_label[0]))
    products_list = recommendations.loc[group_name].Recommendations
    secondary_categories=[]
    for i in products_list:
        category_2nd = products[products.product_name==i].secondary_category
        if str(category_2nd.values[0])=="nan":
            continue        
        elif category_2nd.shape == ():
            secondary_categories.append(category_2nd)
        else:
            secondary_categories.append(category_2nd.iloc[0])
            
    secondary_categories = [k+" ("+str(v)+")" for k,v in {i:secondary_categories.count(i) for i in secondary_categories}.items()]
    print(secondary_categories)
    
    secondary_category = customtkinter.CTkLabel(master=frame, text="Secondary Category:", anchor="w")
    secondary_category.place(x=40,y=120)
    
    secondary_category_options = customtkinter.CTkOptionMenu(master=frame, values=secondary_categories)
    secondary_category_options.place(x=160,y=120)
    
def tertiary_category_filter():
    
    global tertiary_category_options
    
    # Group name
    group_name = int(str(skin_tone_label[0])+str(skin_type_label[0])+str(hair_color_label[0])+str(eye_color_label[0]))
    products_list = recommendations.loc[group_name].Recommendations
    tertiary_categories=[]
    for i in products_list:
        category_3rd = products[products.product_name==i].tertiary_category
        if str(category_3rd.values[0])=="nan":
            continue        
        elif category_3rd.shape == ():
            tertiary_categories.append(category_3rd)
        else:
            tertiary_categories.append(category_3rd.iloc[0])
            
    tertiary_categories = [k+" ("+str(v)+")" for k,v in {i:tertiary_categories.count(i) for i in tertiary_categories}.items()]
    print(tertiary_categories)
    
    tertiary_category = customtkinter.CTkLabel(master=frame, text="Tertiary Category:", anchor="w")
    tertiary_category.place(x=40,y=270)
    
    tertiary_category_options = customtkinter.CTkOptionMenu(master=frame, values=tertiary_categories)
    tertiary_category_options.place(x=160,y=270)

In [28]:
def recommendation_page():
    
    secondary_category = ""
    tertiary_category = ""
    
    if secondary_switch.get():
        secondary_category = secondary_category_options.get()
    if tertiary_switch.get():
        tertiary_category = tertiary_category_options.get()
        secondary_category=""
        
    recommendations = get_recommendations(secondary_category,tertiary_category)
    
    # Clear the screen
    for widget in frame.winfo_children():
        widget.destroy()
    
    root.geometry("500x500")
    
    recommendation_text = customtkinter.CTkLabel(master=frame,text="Here are your product recommendations\n Check the box for products which you like!",font=("Roboto",16))
    recommendation_text.pack(pady=12,padx=10)
    
    global variables
    variables = []

    canvas = customtkinter.CTkCanvas(frame)
    canvas.pack(side="left", fill="both", expand=True)

    scrollbar = tk.Scrollbar(frame, orient="vertical", command=canvas.yview)
    scrollbar.pack(side="right", fill="y")

    canvas.configure(yscrollcommand=scrollbar.set)
    canvas.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))

    inner_frame = customtkinter.CTkFrame(canvas)
    canvas.create_window((0, 0), window=inner_frame, anchor="nw")

    for i, sentence in enumerate(recommendations):
        var = customtkinter.IntVar()
        variables.append(var)
        chk = customtkinter.CTkCheckBox(inner_frame, text=sentence, variable=var)
        chk.pack(anchor="w")
        
    btn_frame = customtkinter.CTkFrame(root)
    btn_frame.pack(fill="x")

    btn = customtkinter.CTkButton(btn_frame, text="Continue", command=thank_you_page)
    btn.pack()

In [27]:
def thank_you_page():
    
    # Clear the screen
    for widget in frame.winfo_children():
        widget.destroy()
        
    display_text = customtkinter.CTkLabel(master=frame,text="Thanks for shopping with us!\n Rate your overall satisfaction!",font=("Roboto",16))
    display_text.pack(pady=12,padx=10)
    
    options = ["Highly satisfied", "Satisfied", "Neutral", "Need a bit improvement", "Not satisfied"]
    var = customtkinter.StringVar(value=options[0])
    
    for option in options:
        rb = customtkinter.CTkRadioButton(frame, text=option, variable=var, value=option)
        rb.pack(pady=12,padx=10,anchor="w")

In [26]:
def get_recommendations(secondary_category,tertiary_category):
    
    # Group name
    group_name = int(str(skin_tone_label[0])+str(skin_type_label[0])+str(hair_color_label[0])+str(eye_color_label[0]))
    products_list = recommendations.loc[group_name].Recommendations
    
    if secondary_category=="" and tertiary_category=="":
        return products_list  
    if secondary_category!="":
        for i in products_list:
            if products[products.product_name==i].secondary_category.values[0]!=secondary_category.split(" ")[0]:
                products_list = products_list[products_list!=i]
        return products_list
    if tertiary_category!="":
        for i in products_list:
            if products[products.product_name==i].tertiary_category.values[0]!=tertiary_category.split(" ")[0]:
                products_list = products_list[products_list!=i]
        return products_list

In [34]:
customtkinter.set_appearance_mode("Light")
customtkinter.set_default_color_theme("dark-blue")

root = customtkinter.CTk()
root.geometry("500x350")

def change_appearance_mode_event(new_appearance_mode: str):
        customtkinter.set_appearance_mode(new_appearance_mode)    

frame = customtkinter.CTkFrame(master=root)
frame.pack(pady=20,padx=60,fill="both",expand=True)

label = customtkinter.CTkLabel(master=frame,text="Welcome to Dummy Sephora",font=("Roboto",24))
label.pack(pady=12,padx=10)

name = customtkinter.CTkEntry(master=frame,placeholder_text="Your name")
name.pack(pady=12,padx=10)

button = customtkinter.CTkButton(master=frame,text="Continue",command=user_page)
button.pack(pady=12,padx=10)

checkbox = customtkinter.CTkCheckBox(master=frame,text="Remember me")
checkbox.pack(pady=12,padx=10)

appearance_mode_label = customtkinter.CTkLabel(master=frame, text="Appearance Mode:", anchor="w")
appearance_mode_label.pack(pady=12,padx=(60,5),side=customtkinter.LEFT)

appearance_mode_optionemenu = customtkinter.CTkOptionMenu(master=frame, values=["Light", "Dark", "System"],command=change_appearance_mode_event)
appearance_mode_optionemenu.pack(pady=12,padx=0,side=customtkinter.LEFT)

root.mainloop()